---
<div align='center'><font size="8" color="#0000FF"> MoA: Statistical Analysis and EDA </font></div>     
<hr> 
<hr>
<img style="float: centre;" src="https://www.rssl.com/~/media/rssl/en/images/sectors/pharmaceutical/pharmaceutical-testing-laboratory-uk.jpg?h=250&la=en&w=250" width="500px"/>
<hr>
<div align='left'><font size="4" color="#0000FF"> Problem Defination</font></div>
<div align='left'><font size="3" color="#000000"> The task of this Mechanisms of Action (MoA) competition is to classify drugs based on multi-labels. The dataset describes the responses of 100 different types of human cells to various drugs. Those response patterns will be used to classify the MoA reponse.This dataset contain multi-label class imbalance problem which is breifly explained in this kerenl. In this kernel I tried to cover almost all the topics related to statistical analysis and eda.
</font></div>
<hr>



# Table of Contents
* [<div align='left'><font size="3" color="#0000FF">1. Importing the necessary libraries</font></div>](#imports)
- [<div align='left'><font size="3" color="#0000FF">2. Reading the datasets</font></div>](#reading)
- [<div align='left'><font size="3" color="#0000FF">3. EDA and Statistical Analysis</font></div>](#eda)
    * <div align='left'><font size="3" color="#0000FF">3.1 Missing Values </font></div>
    * <div align='left'><font size="3" color="#0000FF">3.2 Object datatype</font></div>
        <div align='left'><font size="2.5" color="#0000FF">3.2.1 Descriptive Statistics </font></div>
        <div align='left'><font size="2.5" color="#0000FF">3.2.2 Inferential Statistics </font></div>
        <div align='left'><font size="2.5" color="#0000FF">3.2.3 Visualization </font></div>
    * <div align='left'><font size="3.5" color="#0000FF">3.3 Integer Type</font></div>
         <div align='left'><font size="2.5" color="#0000FF"> 3.3.1 Descriptive Statistics</font></div>
         <div align='left'><font size="2.5" color="#0000FF"> 3.3.2 Outlier Detection</font></div>
         <div align='left'><font size="2.5" color="#0000FF"> 3.3.3 Normality Test</font></div>
         <div align='left'><font size="2.5" color="#0000FF"> 3.3.4 Visualization</font></div>
    * <div align='left'><font size="3" color="#0000FF">3.4 Float Type</font></div>
         <div align='left'><font size="2.5" color="#0000FF"> 3.4.1 Descriptive Statistics</font></div>
         <div align='left'><font size="2.5" color="#0000FF"> 3.4.2 Outlier Detection</font></div>
         <div align='left'><font size="2.5" color="#0000FF"> 3.4.3 Checking the Distribution</font></div>
         <div align='left'><font size="2.5" color="#0000FF"> 3.4.4 Visualization</font></div>
- [<div align='left'><font size="3" color="#0000FF">4. Dimension Reduction </font></div>](#DR)
    * <div align='left'><font size="3" color="#0000FF">4.1 SVD </font></div>
- [<div align='left'><font size="3" color="#0000FF">5. Target Column </font></div>](#model)
    * <div align='left'><font size="3" color="#0000FF">5.1 Multi-label Class Imbalance </font></div>
- [<div align='left'><font size="3" color="#0000FF">6. Conclusion</font></div>](#model) 




<div align='left'><font size="5" color="#0000FF">  Importing necessary Libraries</font></div>     
<hr> 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style('whitegrid')
import statsmodels.api as sm 
from scipy.stats import chi2_contingency,shapiro 
import warnings
warnings.filterwarnings("ignore") 

<div align='left'><font size="4" color="#0000FF"> About Dataset:</font></div>
<div align='left'><font size="3" color="#000000"> Train Data contains 23814 entries and 876 columns, Test Data contains 3982 entries and 876 columns, target column contains 23814 entries and 207 columns. Here our aim is to do statistical analysis on the dataset and find some good insight from it.</font></div>
<hr>
<div align='left'><font size="5" color="#0000FF"> Loading Data</font></div>     
<hr> 


In [ ]:
train = pd.read_csv('../input/lish-moa/train_features.csv')
target = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
test = pd.read_csv('../input/lish-moa/test_features.csv')
ss = pd.read_csv('../input/lish-moa/sample_submission.csv')
train.head()

In [ ]:
print(train.info())
print("---------------------------------------")
print(test.info())
print("---------------------------------------")
print(target.info())

<div align='left'><font size="5" color="#0000FF"> Missing Values</font></div>     
<hr> 

In [ ]:
print("Training missing data:", train.isnull().sum().sum())
print("Testing missing data:", test.isnull().sum().sum())
print("Target missing data:", target.isnull().sum().sum())

<div align='left'><font size="3" color="#000000"> There is no missing values observed in the dataset.</font></div>
<hr>

<div align='left'><font size="5" color="#0000FF">Object Data type</font></div>     
<hr> 
<div align='left'><font size="4.5" color="#0000FF">Descriptive Statistics: Overview of dataset</font></div>     
<hr> 


In [ ]:
print("Train-data")
print(train.describe(include=['O']))
print("-----------------------------------------")
print("Test-data")
print(test.describe(include=['O']))

<div align='left'><font size="5" color="#0000FF">Inferential Statistics : Chi-square (χ2) test for independence</font></div>     
<hr>
<div align='left'><font size="3" color="#000000"> Chi-square test is a non-parametric method used to compare the relationship between the two categorical (nominal) variables in a contingency table.</font></div>
<div align='left'><font size="3" color="#000000">Null hypotheses(H0): The two categorical variables are independent (no association between the two variables).</font></div>
<div align='left'><font size="3" color="#000000">Alternative hypotheses(H1): The two categorical variables are dependent (there is an association between the two variables)</font></div>
<hr>


In [ ]:
pd.crosstab(train['cp_type'],train['cp_dose'])

In [ ]:
print('Chi square Test ')
print('H0: Variables are Independent')
print('H0: Variables are dependent \n')
data = pd.crosstab(train['cp_type'],train['cp_dose'])
stat, p, dof, expected = chi2_contingency(data) 
alpha = 0.05
print("p value:", p) 
if p <= alpha: 
    print('Conclusion: Variables are dependent(reject H0)') 
else: 
    print('Conclusion: Variables are Independent(H0 holds true)') 

<div align='left'><font size="3" color="#000000">The P-value obtained from chi-square test for independence is not significant (P > 0.05), and therefore, we conclude that there is no significant association between variable.</font></div>
<hr>

<div align='left'><font size="4.5" color="#0000FF"> Visualization</font></div>     
<hr>



In [ ]:
f,ax = plt.subplots(1,3,figsize=(18,5))
sns.countplot('cp_type',data=train,ax=ax[0])
sns.countplot('cp_dose',data=train,ax=ax[1])
sns.countplot('cp_type',hue='cp_dose',data=train,ax=ax[2],palette='husl')
ax[0].set_title('cp_type')
ax[1].set_title('cp_dose')
ax[2].set_title('Counter plot of cp_dose vs cp_dose')


<div align='left'><font size="5" color="#0000FF"> Integer Data type</font></div>     
<hr>
<div align='left'><font size="4.5" color="#0000FF"> Descriptive Statistics: Overview of dataset</font></div>     
<hr>


In [ ]:
print("Train-data")
print(train.select_dtypes('int').describe().transpose())
print("--------------------------------------------------------------------")
print("Test-data")
print(test.select_dtypes('int').describe().transpose())

In [ ]:
print("Train-data")
print(train.select_dtypes('int').agg(['var','std','kurt','skew']).transpose())
print("---------------------------------------------------------")
print("Test-data")
print(test.select_dtypes('int').agg(['var','std','kurt','skew']).transpose())

<div align='left'><font size="5" color="#0000FF"> Outlier Detection : IQR method</font></div>     
<hr>
<div align='left'><font size="3" color="#000000"> In this method outliers detected using Inter Quartile Range(IQR). For more details visit this
   <a href="https://www.kaggle.com/nareshbhat/outlier-the-silent-killer">link.</a> </font></div>


In [ ]:
out=[]
def iqr_outliers(df):
    for i in df:
        q1 = df.quantile(0.25)
        q3 = df.quantile(0.75)
        iqr = q3-q1
        Lower_tail = q1 - 1.5 * iqr
        Upper_tail = q3 + 1.5 * iqr
        if i > Upper_tail or i < Lower_tail:
            out.append(i)
    print("Outliers:",out)
iqr_outliers(train['cp_time'])

<div align='left'><font size="3" color="#000000"> We Find that there is no outlier. To understand better we use normality test (Shapiro Wilk test).</font></div>
<hr> 

<div align='left'><font size="5" color="#0000FF"> Inferential Statistics : Shapiro Wilk test</font></div>     
<hr>

<div align='left'><font size="3" color="#000000"> Shapiro Wilk test is method used to Check the whether the data is normally distributed or not. </font></div>
<div align='left'><font size="3" color="#000000"> Null hypotheses(H0): Distribution is not Normal.</font></div>
<div align='left'><font size="3" color="#000000">  Alternative hypotheses(H1): Distribution is Normal.</font></div>
<hr>


In [ ]:
print('Shapiro Wilk test')
print('H0: Distribution is not Normal.')
print('H0:Distribution is Normal. \n')
stat, p = shapiro(test.select_dtypes('int'))
alpha = 0.05
print("p value:", p) 
if p <= alpha: 
    print('Conclusion: Distribution is Normal(reject H0)') 
else: 
    print('Conclusion: Distribution is not Normal(H0 holds true)') 

<div align='left'><font size="5" color="#0000FF"> Float Data type</font></div>     
<hr>
<div align='left'><font size="4.5" color="#0000FF"> Visualization</font></div>     
<hr>


In [ ]:
f,ax = plt.subplots(1,3,figsize=(18,5))
sns.distplot(train['cp_time'],ax=ax[0])
sns.boxplot(data=train['cp_time'],ax=ax[1])
sm.qqplot(train['cp_time'], line ='45',ax=ax[2]) 
ax[0].set_title('Distribution Plot')
ax[1].set_title('Box  Plot')
ax[2].set_title('QQ Plot')

<div align='left'><font size="5" color="#0000FF"> Float Data type</font></div>     
<hr>
<div align='left'><font size="4" color="#0000FF"> Descriptive Statistics: Overview of dataset</font></div>     
<hr>


In [ ]:
print("Train-data")
print(train.select_dtypes('float').describe().transpose())
print("--------------------------------------------------------------------")
print("Test-data")
print(test.select_dtypes('float').describe().transpose())

In [ ]:
print("Train-data")
print(train.select_dtypes('float').agg(['var','std','kurt','skew']).transpose())
print("---------------------------------------------------------")
print("Test-data")
print(test.select_dtypes('float').agg(['var','std','kurt','skew']).transpose())

<div align='left'><font size="3" color="#000000">  As the dataset is very huge we take 5 variable randomly to analyze it.</font></div>
<hr>

<div align='left'><font size="5" color="#0000FF"> Outlier Detection : IQR method</font></div>     
<hr>

<div align='left'><font size="3" color="#000000"> In this method outliers detected using Inter Quartile Range(IQR). For more details visit this
   <a href="https://www.kaggle.com/nareshbhat/outlier-the-silent-killer">link.</a> </font></div>

In [ ]:
random_train = train.select_dtypes('float').sample(n=5, axis=1,random_state=123)
random_train.head()

In [ ]:
print('List of Outliers')
out1=[]
def iqr_outliers(df):
    for i in df:
        q1 = df.quantile(0.25)
        q3 = df.quantile(0.75)
        iqr = q3-q1
        Lower_tail = q1 - 1.5 * iqr
        Upper_tail = q3 + 1.5 * iqr
        if i > Upper_tail or i < Lower_tail:
            out1.append(i)
    print("Outliers:",out1)
iqr_outliers(train['c-15'])


<div align='left'><font size="5" color="#0000FF"> Inferential Statistics : Shapiro Wilk test</font></div>     
<hr>

<div align='left'><font size="3" color="#000000">  Null hypotheses(H0): Distribution is not Normal.</font></div>
<div align='left'><font size="3" color="#000000">  Alternative hypotheses(H1): Distribution is Normal.</font></div>
<hr>


In [ ]:
print('Shapiro Wilk test')
print('H0: Distribution is not Normal.')
print('H0:Distribution is Normal. \n')
def shapiro_wilk(df,col):
  print('Column Name:',col)
  stat, p = shapiro(df[col])
  alpha = 0.05
  print("p value:", p) 
  if p <= alpha: 
    print('Conclusion: Distribution is Normal(reject H0)') 
  else: 
    print('Conclusion: Distribution is not Normal(H0 holds true)') 
shapiro_wilk(train,'g-523')
shapiro_wilk(train,'g-731')
shapiro_wilk(train,'g-180')
shapiro_wilk(train,'g-185')
shapiro_wilk(train,'g-435')

As the dataset is large, variable is tending to normal. In other words as the sample size increases, the standard deviation of the means decreases([Central limit theorem](https://demonstrations.wolfram.com/DistributionOfNormalMeansWithDifferentSampleSizes/#:~:text=The%20population%20mean%20of%20the,the%20distribution%20being%20sampled%20from.&text=Thus%20as%20the%20sample%20size,of%20the%20sample%20means%20increases.)). We conclude that all the variable is normally distributed.

<div align='left'><font size="5" color="#0000FF"> Visualization</font></div>     
<hr>



In [ ]:
f,ax = plt.subplots(3,5,figsize=(20,12))
sns.distplot(train['g-523'],ax=ax[0,0])
sns.distplot(train['g-731'],ax=ax[0,1])
sns.distplot(train['g-180'],ax=ax[0,2])
sns.distplot(train['g-185'],ax=ax[0,3])
sns.distplot(train['g-435'],ax=ax[0,4])
sns.boxplot(data=train['g-523'],ax=ax[1,0])
sns.boxplot(data=train['g-731'],ax=ax[1,1])
sns.boxplot(data=train['g-180'],ax=ax[1,2])
sns.boxplot(data=train['g-185'],ax=ax[1,3])
sns.boxplot(data=train['g-435'],ax=ax[1,4])
sm.qqplot(train['g-523'], line ='45',ax=ax[2,0]) 
sm.qqplot(train['g-731'], line ='45',ax=ax[2,1]) 
sm.qqplot(train['g-180'], line ='45',ax=ax[2,2]) 
sm.qqplot(train['g-185'], line ='45',ax=ax[2,3]) 
sm.qqplot(train['g-435'], line ='45',ax=ax[2,4]) 

ax[0,0].set_title('Distribution Plot of g-523')
ax[0,1].set_title('Distribution Plot of g-731')
ax[0,2].set_title('Distribution Plot of g-180')
ax[0,3].set_title('Distribution Plot of g-185')
ax[0,4].set_title('Distribution Plot of g-435')
ax[1,0].set_title('Box Plot of g-523')
ax[1,1].set_title('Box Plot of g-731')
ax[1,2].set_title('Box Plot of g-180')
ax[1,3].set_title('Box Plot of g-185')
ax[1,4].set_title('Box Plot of g-435')
ax[2,0].set_title('QQ Plot of g-523')
ax[2,1].set_title('QQ Plot of g-731')
ax[2,2].set_title('QQ Plot of g-180')
ax[2,3].set_title('QQ Plot of g-185')
ax[2,4].set_title('QQ Plot of g-435')

In [ ]:
X_train   = train.drop(['sig_id'],axis=1)
y_train   = target.drop(['sig_id'],axis=1)
X_test    = test.drop(['sig_id'],axis=1)
target    = target.drop(['sig_id'],axis=1)
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

<div align='left'><font size="5" color="#0000FF"> Dimensional Reduction</font></div>     
<hr>

<div align='left'><font size="3" color="#000000"> As the dataset is very huge, It is difficult to train the model.Here Dimensionality Reduction technique play an important rule that reduce the training time and remove unwanted columns. Here Singular Value decomposition is applied. 
</font></div>
<hr>

<div align='left'><font size="4.5" color="#0000FF"> Singular Value decomposition(SVD):</font></div>     
<hr>
<div align='left'><font size="3" color="#000000"> SVD is a matrix factorization technique where a matrix is decomposed into a product of a square matrix, a diagonal matrix, and another square matrix. It is used widely both in the calculation of other matrix operations, such as matrix inverse, but also as a data reduction in machine learning. Here we reduce the dimension to 400.
</font></div>


In [ ]:
from sklearn.decomposition import TruncatedSVD
svd =  TruncatedSVD(n_components = 400)
dim_reduction = pd.DataFrame(svd.fit_transform(X_train))
dim_reduction.head()

<div align='left'><font size="5" color="#0000FF"> Target Column: Multi-Label Classification</font></div>     
<hr>

<div align='left'><font size="3" color="#000000"> Multi-Label Classification is a classification problem where target variable has more than one dimension and each dimension is binary. 
</font></div>
<hr>

> <img style="float: centre;" src="https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQUqgAcgROPlnULCx5vbIvHAFyMP7DO0he9OQ&usqp=CAU" width="450px"/>


In [ ]:
target.info()

In [ ]:
target.head()

<div align='left'><font size="5" color="#0000FF"> Multi-label Data Imbalance Problem</font></div>     
<hr>

<div align='left'><font size="3" color="#000000"> Class Imbalance is one of the most common problems in classification, which occurs in many of the real-world datasets. This class imbalance can create a challenge for predictive modelling tasks and lead to poor predictive performance for minority class, as most of the machine learning algorithms are developed with the assumption of the class is balanced. Handling the imbalance class will create a balanced class distribution and remove the bias towards minority classes.
</font></div>
<hr>
<img style="float: centre;" src="https://miro.medium.com/max/3200/1*LMqkFEL6uE5JPEcYgjmN4g.jpeg" width="450px"/>
<hr>
<div align='left'><font size="3" color="#000000"> Here the task is multi label classification where the target variable has more than one dimension where each dimension is binary. In gerneral to solve imbalance problem there are 3 major techniques are available namely random upsampling,random downsampling,SMOTE. But these methods are holds good only when label is in one dimension(binary/multi-label not multiclass). MLSMOTE is the technique to deal with multi label data imbalance problem. Unfortunately there is no package available in python to solve this problem.This Research Paper gives a high level idea about MLSMOTE ➜ <a href="https://www.sciencedirect.com/science/article/abs/pii/S0950705115002737">MLSMOTE: Approaching imbalanced multilabel learning through synthetic instance generation</a>
</font></div>


<div align='left'><font size="5" color="#0000FF"> Conclusion</font></div>     
<hr>
<div align='left'><font size="3" color="#000000">1. EDA helpful to understand the overview of the dataset. </font></div>   
<div align='left'><font size="3" color="#000000">2. Chi Square test tells there is no significant relationship between variable of object type.</font></div> 
<div align='left'><font size="3" color="#000000">3. No outlier is found in integer datatype columns and float datatype columns contain outliers.</font>
<div align='left'><font size="3" color="#000000">4. Shapiro-Wilk test gives the conclusion that distribution is normal.</font></div> 
<div align='left'><font size="3" color="#000000">5. This dataset facing Multi-label Data Imbalance Problem.</font></div> 
<hr>
    

<div align='left'><font size="5" color="#0000FF"> References</font></div>
<hr>
<div align='left'><font size="3"><a href="https://www.analyticsvidhya.com/blog/2017/08/introduction-to-multi-label-classification/" target="_blank">1.Solving Multi-Label Classification problems</a></div>
<hr>
<div align='left'><font size="3"><a href="https://medium.com/thecyphy/handling-data-imbalance-in-multi-label-classification-mlsmote-531155416b87" target="_blank">2.Handling Data Imbalance in Multi-label Classification</a></div>
<hr>  
<div align='left'><font size="3"><a href="https://www.kaggle.com/rhodiumbeng/classifying-multi-label-comments-0-9741-lb" target="_blank">3.Classifying multi-label comments</a> </div>
<hr>  
<div align='left'><font size="3"><a href="https://www.sciencedirect.com/science/article/abs/pii/S0950705115002737" target="_blank">4.MLSMOTE: Approaching imbalanced multilabel learning through synthetic instance generation</a></div>
    


<div class="alert alert-info">
  <strong></strong><font size="5"> In this kernel I tried to cover almost all the topics related to statistical analysis and eda.Feel free to ask any question related to this topic. I'm happy to answer.
</div>
<hr>
<div class="alert alert-success"><font size="5">
   If you like my work Kindly <strong>UPVOTE!</strong> HAPPY LEARNING :)
</div>
